# 中級：将来予測・施策提案（AI を用いた示唆生成）

この章では、過去データを基に LLM に施策提案や未来仮説を生成させて、  
データの傾向分析に「戦略思考」を加える練習をします。

ステップ概要：
1. 分析対象カテゴリや属性ターゲットを決める  
2. それらについて、集計傾向を可視化／確認  
3. プロンプト作成：過去傾向 + 質問形式で LLM に投げる  
4. ai_query による施策生成  
5. 生成案を結果テーブルに紐づけてレビュー可能な形に  

In [0]:
%pip install markdown
dbutils.library.restartPython()

In [0]:
CATALOG = "hiroshi"
SCHEMA  = "survey_analysis_simple"

# UCオブジェクト作成
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {SCHEMA}")

GOLD_TABLE_NAME = "gold_survey_responses_final"

## 必要モジュール読み込みとデータ取得

このセルでは、クラスタリングに必要な SparkML モジュールをインポートし、  
GOLD テーブルを取得してクラスタリング対象データを準備します。

In [0]:
gold = spark.table(GOLD_TABLE_NAME)  # ← 実際の GOLD テーブル名
display(gold.limit(10))
gold.printSchema()

## 対象カテゴリ／属性ターゲットの傾向確認

このセルでは、まずどのカテゴリがこれまで多く言及されたかを確認して、どのカテゴリを将来提案対象にするかを決めやすくします。

例：ネガティブカテゴリー別出現件数の上位 5 カテゴリを見て、それを将来改善候補として洗い出します。

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

neg_dist = (
    gold
    .groupBy("negative_feedback_category")
    .agg(F.count("*").alias("cnt"))
    .orderBy(F.desc("cnt"))
)

display(neg_dist.limit(5))

## 将来テーマ候補カテゴリを選出し、プロンプト対象化

このセルでは、「将来改善すべきカテゴリ」候補を選び（たとえば出現頻度上位 3〜5 業種）、  
それを LLM に渡すためのプロンプト文テンプレートを準備します。

In [0]:
from pyspark.sql.functions import lit, concat, col, format_string

# 上位のカテゴリ名リストを取得（collect 等で Python 側に取り出さない、Spark 上で処理）
top_neg_df = neg_dist.limit(3).select("negative_feedback_category")

# テンプレート（Spark の format_string を使って置換できる形式にする）
prompt_tpl_spark = (
    "過去アンケート調査から、このカテゴリ「%s」に関する不満・改善要望が多く見られました。"
    "この傾向をもとに、今後1〜2年で最も有効と思われる **改善施策案を3つ**、およびその理由を説明して、マークダウン形式で出力してください。"
    "\n\n対象カテゴリ: \"%s\""
)

# prompt 列を作成（2回 %s を使ってカテゴリ名を埋め込む）
df_prompt = top_neg_df.withColumn(
    "prompt",
    format_string(prompt_tpl_spark, col("negative_feedback_category"), col("negative_feedback_category"))
).withColumnRenamed("negative_feedback_category", "neg_cat")

display(df_prompt)

## ai_query により施策提案・予測案を取得

このセルでは、先に作った `prompt` 列を使って `ai_query` を呼び出し、  
施策案（3案）＋理由を出す説明文を取得します（構造化出力を期待する）。

In [0]:
# セル 3：LLM から施策案を文字列で取得（text モード）
df_out = df_prompt.selectExpr(
    "neg_cat",
    "ai_query('databricks-gpt-oss-120b', prompt, responseFormat => '{\"type\":\"text\"}') as strategy_plan"
)

display(df_out)

### カテゴリ別施策案をダッシュボード風に表示

このセルでは、セル 3 で生成されたカテゴリ別施策案（`strategy_plan`）を使って、  
受講者が施策案を視覚的に理解しやすい形で表示します。

具体的には、以下のような工夫を入れます：

- 各カテゴリ × 施策案を整形して見やすく表示  
- 必要なら HTML テーブル風、または Spark の `display()` を使ってリンク付き表示  
- 施策案が複数行にまたがるなら `split()` や改行で分割して列展開  

この表示を使えば、「どのカテゴリに、どのような施策案が出たか」が一目で分かるようになります。

In [0]:
from markdown import markdown  # `%pip install markdown` 必要かも
from IPython.display import display, HTML

def show_markdown(md: str):
    html = markdown(md)
    display(HTML(html))

# 使う例
for row in df_out.collect():
    cat = row["neg_cat"]
    plan_md = row["strategy_plan"]
    md = f"# カテゴリ：{cat}\n\n" + plan_md
    show_markdown(md)
    show_markdown("---")

---

## 🏁 総括（まとめ）

### 1. 実施プロセスの振り返り  
本演習では、セル 3 において各ネガティブカテゴリごとに「今後1〜2年で打つべき改善施策案を3案ずつ」生成し、実施内容と期待効果・根拠を併記いたしました。  
さらにセル 4 では、それらの施策案をカテゴリ別に見やすく整理・表示するよう設計し、受講者が直感的に比較・参照できる形式を追求いたしました。

### 2. 各カテゴリに対する提案の特徴（例：MouthfeelCarbon-ation）  
- **炭酸圧力と溶解度の最適化**：ラインのリアルタイムモニタリングおよび高精度制御技術導入により、炭酸感の一貫性を強化  
- **微細気泡技術の導入**：気泡径を縮小・制御することで、舌触りの滑らかさ・炭酸感持続性を向上  
- **感覚データ駆動のレシピ最適化**：消費者パネルと AI を組み合わせ、味覚パラメータの最適調整を行う手法  

各案には「具体性」と「実現可能性」が示され、不満要因への対応力が明確に説明されています。

### 3. 可視化表示による利点と工夫  
- テーブル形式または行展開形式で一覧表示することで、カテゴリ × 施策案の構造を視覚化  
- HTML 表示や行番号付き形式を用いることで、報告資料や受講資料としての可読性を向上  
- 表示形式により、施策の偏り・重複・カテゴリ間比較といった構造的な課題を発見しやすくなります

### 4. 今後の発展方向  
- 各案に対して **スコアリング（効果予測値／優先度指標）** を付与し、意思決定精度を高める  
- 属性別（年代・購入頻度等）に施策案を分岐させ、パーソナライズされた提案を可能とする  
- 施策案を BI ダッシュボードとして展開し、リアルタイム参照・更新体制を構築  
- 生成案を題材とした受講者演習（改善案改良・代替案提示）を設計し、思考深化に用いる

### 🎯 最終メッセージ  
本ワークフロー構成により、単なる分析や施策生成で終わることなく、**見やすく整理された可視化付き施策案出力**までを含む一連の流れを実現しました。  